# Library

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

import warnings
warnings.filterwarnings(action='ignore')

# GPU 용량 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:  # gpu가 있다면, 용량 한도를 5GB로 설정
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3*1024)])

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# 최적 모델 찾기 (seed = 42)
- [최적 모델 찾기](https://snowdeer.github.io/machine-learning/2018/01/09/find-best-model/)
- 모델 생성 시 seed = 42 ~ 50일 때, 각각의 best모델에서 data seed = 1~50에서의 평균 정확도
- 모델 생성 시 seed = 42, 평균 정확도 : 82.6
- 모델 생성 시 seed = 43, 평균 정확도 : 81.6
- 모델 생성 시 seed = 44, 평균 정확도 : 82.5
- 모델 생성 시 seed = 45, 평균 정확도 : 82.5
- 모델 생성 시 seed = 46, 평균 정확도 : 81.6
- 모델 생성 시 seed = 47, 평균 정확도 : 81.8
- 모델 생성 시 seed = 48, 평균 정확도 : 81.1
- 모델 생성 시 seed = 49, 평균 정확도 : 82.3
- 모델 생성 시 seed = 50, 평균 정확도 : 80

## seed별 best model 저장

In [43]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


import random
# ----------------------
seed_num = 50
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# ---------------------
seed_num = 50 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'seed'f'{seed_num}''-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_acc', patience=30, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 5s 64ms/step - loss: 0.6754 - acc: 0.5949 - val_loss: 0.6753 - val_acc: 0.5970

Epoch 00001: val_loss improved from inf to 0.67535, saving model to ./model/seed50-01-0.6753.hdf5
Epoch 2/500
37/37 [==============================] - 2s 52ms/step - loss: 0.6743 - acc: 0.6031 - val_loss: 0.6701 - val_acc: 0.5970

Epoch 00002: val_loss improved from 0.67535 to 0.67009, saving model to ./model/seed50-02-0.6701.hdf5
Epoch 3/500
37/37 [==============================] - 2s 52ms/step - loss: 0.6285 - acc: 0.6398 - val_loss: 0.5877 - val_acc: 0.7025

Epoch 00003: val_loss improved from 0.67009 to 0.58767, saving model to ./model/seed50-03-0.5877.hdf5
Epoch 4/500
37/37 [==============================] - 2s 52ms/step - loss: 0.5010 - acc: 0.7681 - val_loss: 0.5157 - val_acc: 0.7536

Epoch 00004: val_loss improved from 0.58767 to 0.51568, saving model to ./model/seed50-04-0.5157.hdf5
Epoch 5/500
37/37 [==============================] - 2s 52ms/ste

In [44]:
# ----------------------------
seed_num = 50
# ----------------------------

from keras.models import load_model
best_model = load_model('./model/seed50-04-0.5157.hdf5') # 수동으로 확인

dic={}
for seed in range(0, 50):
    random.seed(seed)

    x = np.load('/project/LSH/x_(7727,10,4069).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')
    
df = pd.DataFrame.from_dict(dic, orient='index')
print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.7870550161812297, seed_num = 0
정확도 :0.8019417475728156, seed_num = 1
정확도 :0.7954692556634304, seed_num = 2
정확도 :0.8090614886731392, seed_num = 3
정확도 :0.7954692556634304, seed_num = 4
정확도 :0.8025889967637541, seed_num = 5
정확도 :0.7954692556634304, seed_num = 6
정확도 :0.7877022653721683, seed_num = 7
정확도 :0.8084142394822007, seed_num = 8
정확도 :0.8006472491909385, seed_num = 9
정확도 :0.8129449838187702, seed_num = 10
정확도 :0.8129449838187702, seed_num = 11
정확도 :0.8064724919093851, seed_num = 12
정확도 :0.8116504854368932, seed_num = 13
정확도 :0.8045307443365696, seed_num = 14
정확도 :0.8019417475728156, seed_num = 15
정확도 :0.8116504854368932, seed_num = 16
정확도 :0.7902912621359224, seed_num = 17
정확도 :0.8006472491909385, seed_num = 18
정확도 :0.8019417475728156, seed_num = 19
정확도 :0.8071197411003236, seed_num = 20
정확도 :0.7909385113268609, seed_num = 21
정확도 :0.7941747572815534, seed_num = 22
정확도 :0.7922330097087379, seed_num = 23
정확도 :0.7967637540453074, seed_num = 24
정확도 :0.8148867313915857, seed_num =

# (79.9) LSTM / all fit / earlystop
- 22-02-23

In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

# ---------------------
seed_num = 42 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'allfit3_seed42-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(x, y, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
46/46 [==============================] - 6s 76ms/step - loss: 0.6458 - acc: 0.6511 - val_loss: 0.6934 - val_acc: 0.4865

Epoch 00001: val_loss improved from inf to 0.69343, saving model to ./model/allfit3_seed42-01-0.6934.hdf5
Epoch 2/500
46/46 [==============================] - 3s 65ms/step - loss: 0.6740 - acc: 0.6066 - val_loss: 0.7237 - val_acc: 0.4865

Epoch 00002: val_loss did not improve from 0.69343
Epoch 3/500
46/46 [==============================] - 3s 66ms/step - loss: 0.6651 - acc: 0.6419 - val_loss: 0.7362 - val_acc: 0.4865

Epoch 00003: val_loss did not improve from 0.69343
Epoch 4/500
46/46 [==============================] - 3s 66ms/step - loss: 0.6369 - acc: 0.6518 - val_loss: 0.6848 - val_acc: 0.4865

Epoch 00004: val_loss improved from 0.69343 to 0.68479, saving model to ./model/allfit3_seed42-04-0.6848.hdf5
Epoch 5/500
46/46 [==============================] - 3s 65ms/step - loss: 0.5325 - acc: 0.7372 - val_loss: 0.5727 - val_acc: 0.7091

Epoch 00005: val_


Epoch 00040: val_loss did not improve from 0.55251
Epoch 41/500
46/46 [==============================] - 3s 67ms/step - loss: 0.0850 - acc: 0.9789 - val_loss: 1.0211 - val_acc: 0.7308

Epoch 00041: val_loss did not improve from 0.55251
Epoch 42/500
46/46 [==============================] - 3s 66ms/step - loss: 0.0824 - acc: 0.9800 - val_loss: 1.0499 - val_acc: 0.7293

Epoch 00042: val_loss did not improve from 0.55251
Epoch 43/500
46/46 [==============================] - 3s 67ms/step - loss: 0.0812 - acc: 0.9822 - val_loss: 1.0668 - val_acc: 0.7241

Epoch 00043: val_loss did not improve from 0.55251
Epoch 44/500
46/46 [==============================] - 3s 67ms/step - loss: 0.0803 - acc: 0.9821 - val_loss: 1.0820 - val_acc: 0.7262

Epoch 00044: val_loss did not improve from 0.55251
Epoch 45/500
46/46 [==============================] - 3s 67ms/step - loss: 0.0777 - acc: 0.9829 - val_loss: 1.0839 - val_acc: 0.7298

Epoch 00045: val_loss did not improve from 0.55251
Epoch 46/500
46/46 [===

In [2]:
from sklearn import metrics 

import random
# ----------------------------
seed_num = 42
# ----------------------------

with tf.device('/device:GPU:0'):

    from keras.models import load_model
    best_model = load_model('./model/allfit3_seed42-06-0.5525.hdf5') # 수동으로 확인

    dic={}
    for seed in range(0, 50):
        random.seed(seed)

        x = np.load('/project/LSH/x_(7727,10,4068).npy')
        y = np.load('/project/LSH/y_(7727,1).npy')

        idx = list(range(len(x)))
        random.shuffle(idx)

        i = round(x.shape[0]*0.8)
        X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
        X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

        pred = best_model.predict(X_test)
        pred[pred>0.5]=1
        pred[pred<=0.5]=0
        acc = metrics.accuracy_score(y_test, pred)
        dic[seed]=acc
        print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

    df = pd.DataFrame.from_dict(dic, orient='index')
    print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

Using TensorFlow backend.


정확도 :0.7980582524271844, seed_num = 0
정확도 :0.8032362459546926, seed_num = 1
정확도 :0.7948220064724919, seed_num = 2
정확도 :0.8064724919093851, seed_num = 3
정확도 :0.8006472491909385, seed_num = 4
정확도 :0.7974110032362459, seed_num = 5
정확도 :0.8025889967637541, seed_num = 6
정확도 :0.7948220064724919, seed_num = 7
정확도 :0.8116504854368932, seed_num = 8
정확도 :0.7922330097087379, seed_num = 9
정확도 :0.8077669902912621, seed_num = 10
정확도 :0.8084142394822007, seed_num = 11
정확도 :0.8084142394822007, seed_num = 12
정확도 :0.8045307443365696, seed_num = 13
정확도 :0.801294498381877, seed_num = 14
정확도 :0.8032362459546926, seed_num = 15
정확도 :0.8051779935275081, seed_num = 16
정확도 :0.7799352750809061, seed_num = 17
정확도 :0.8019417475728156, seed_num = 18
정확도 :0.8071197411003236, seed_num = 19
정확도 :0.7896440129449838, seed_num = 20
정확도 :0.7818770226537217, seed_num = 21
정확도 :0.7928802588996764, seed_num = 22
정확도 :0.7844660194174757, seed_num = 23
정확도 :0.7961165048543689, seed_num = 24
정확도 :0.8122977346278317, seed_num = 

# () LSTM / all fit / no earlystop

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


import random

with tf.device('/device:GPU:0'):
    # ----------------------
    seed_num = 42
    # ----------------------
    random.seed(seed_num)

    x = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    # ---------------------
    seed_num = 42 
    # ---------------------
    tf.random.set_seed(seed_num)

    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))

    from tensorflow.keras.callbacks import ModelCheckpoint
    import os

    MODEL_SAVE_FOLDER_PATH = './model/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    model_path = MODEL_SAVE_FOLDER_PATH + 'lstm_allfit_noearly-{epoch:02d}-{val_loss:.4f}.hdf5'

    cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                    verbose=1, save_best_only=True)

    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
    lstm.fit(x, y, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[cb_checkpoint], shuffle=False)

Epoch 1/500
46/46 [==============================] - 8s 88ms/step - loss: 0.6458 - acc: 0.6511 - val_loss: 0.6934 - val_acc: 0.4865

Epoch 00001: val_loss improved from inf to 0.69343, saving model to ./model/lstm_allfit_noearly-01-0.6934.hdf5
Epoch 2/500
46/46 [==============================] - 3s 75ms/step - loss: 0.6740 - acc: 0.6066 - val_loss: 0.7237 - val_acc: 0.4865

Epoch 00002: val_loss did not improve from 0.69343
Epoch 3/500
46/46 [==============================] - 4s 79ms/step - loss: 0.6651 - acc: 0.6419 - val_loss: 0.7362 - val_acc: 0.4865

Epoch 00003: val_loss did not improve from 0.69343
Epoch 4/500
46/46 [==============================] - 4s 78ms/step - loss: 0.6369 - acc: 0.6518 - val_loss: 0.6848 - val_acc: 0.4865

Epoch 00004: val_loss improved from 0.69343 to 0.68478, saving model to ./model/lstm_allfit_noearly-04-0.6848.hdf5
Epoch 5/500
46/46 [==============================] - 4s 77ms/step - loss: 0.5325 - acc: 0.7367 - val_loss: 0.5728 - val_acc: 0.7091

Epoch 0

In [ ]:
from sklearn import metrics 

import random
# ----------------------------
seed_num = 42
# ----------------------------

with tf.device('/device:GPU:0'):

    from keras.models import load_model
    best_model = load_model('./model/') # 수동으로 확인

    dic={}
    for seed in range(0, 50):
        random.seed(seed)

        x = np.load('/project/LSH/x_(7727,10,4068).npy')
        y = np.load('/project/LSH/y_(7727,1).npy')

        idx = list(range(len(x)))
        random.shuffle(idx)

        i = round(x.shape[0]*0.8)
        X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
        X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

        pred = best_model.predict(X_test)
        pred[pred>0.5]=1
        pred[pred<=0.5]=0
        acc = metrics.accuracy_score(y_test, pred)
        dic[seed]=acc
        print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

    df = pd.DataFrame.from_dict(dic, orient='index')
    print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

# (82.6) LSTM / not all fit / earlystop

In [6]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# ---------------------
seed_num = 42 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'seed42-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 5s 64ms/step - loss: 0.6757 - acc: 0.5984 - val_loss: 0.6654 - val_acc: 0.6177

Epoch 00001: val_loss improved from inf to 0.66544, saving model to ./model/seed42-01-0.6654.hdf5
Epoch 2/500
37/37 [==============================] - 2s 52ms/step - loss: 0.6690 - acc: 0.6089 - val_loss: 0.6482 - val_acc: 0.6177

Epoch 00002: val_loss improved from 0.66544 to 0.64818, saving model to ./model/seed42-02-0.6482.hdf5
Epoch 3/500
37/37 [==============================] - 2s 52ms/step - loss: 0.6000 - acc: 0.6713 - val_loss: 0.5525 - val_acc: 0.7354

Epoch 00003: val_loss improved from 0.64818 to 0.55246, saving model to ./model/seed42-03-0.5525.hdf5
Epoch 4/500
37/37 [==============================] - 2s 52ms/step - loss: 0.4867 - acc: 0.7793 - val_loss: 0.5088 - val_acc: 0.7613

Epoch 00004: val_loss improved from 0.55246 to 0.50875, saving model to ./model/seed42-04-0.5088.hdf5
Epoch 5/500
37/37 [==============================] - 2s 52ms/ste

37/37 [==============================] - 2s 53ms/step - loss: 0.0637 - acc: 0.9875 - val_loss: 1.1007 - val_acc: 0.7264

Epoch 00044: val_loss did not improve from 0.50213
Epoch 45/500
37/37 [==============================] - 2s 53ms/step - loss: 0.0626 - acc: 0.9871 - val_loss: 1.1301 - val_acc: 0.7290

Epoch 00045: val_loss did not improve from 0.50213
Epoch 46/500
37/37 [==============================] - 2s 52ms/step - loss: 0.0666 - acc: 0.9860 - val_loss: 1.0977 - val_acc: 0.7309

Epoch 00046: val_loss did not improve from 0.50213
Epoch 47/500
37/37 [==============================] - 2s 52ms/step - loss: 0.0645 - acc: 0.9860 - val_loss: 1.1112 - val_acc: 0.7264

Epoch 00047: val_loss did not improve from 0.50213
Epoch 48/500
37/37 [==============================] - 2s 52ms/step - loss: 0.0662 - acc: 0.9853 - val_loss: 1.1206 - val_acc: 0.7283

Epoch 00048: val_loss did not improve from 0.50213
Epoch 49/500
37/37 [==============================] - 2s 52ms/step - loss: 0.0605 - acc:

In [9]:
from keras.models import load_model
best_model = load_model('./model/seed42-05-0.5021.hdf5') 

dic_42={}
for seed in range(0, 50):
    random.seed(seed)

    x = np.load('/project/LSH/x_(7727,10,4069).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic_42[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

정확도 :0.8194174757281554, seed_num = 0
정확도 :0.8362459546925566, seed_num = 1
정확도 :0.8207119741100324, seed_num = 2
정확도 :0.8388349514563107, seed_num = 3
정확도 :0.8252427184466019, seed_num = 4
정확도 :0.8401294498381877, seed_num = 5
정확도 :0.8103559870550162, seed_num = 6
정확도 :0.8155339805825242, seed_num = 7
정확도 :0.827831715210356, seed_num = 8
정확도 :0.8317152103559871, seed_num = 9
정확도 :0.8265372168284789, seed_num = 10
정확도 :0.8310679611650486, seed_num = 11
정확도 :0.8349514563106796, seed_num = 12
정확도 :0.8343042071197411, seed_num = 13
정확도 :0.8381877022653722, seed_num = 14
정확도 :0.8336569579288026, seed_num = 15
정확도 :0.8330097087378641, seed_num = 16
정확도 :0.8174757281553398, seed_num = 17
정확도 :0.8271844660194175, seed_num = 18
정확도 :0.8388349514563107, seed_num = 19
정확도 :0.8284789644012945, seed_num = 20
정확도 :0.8142394822006472, seed_num = 21
정확도 :0.8226537216828479, seed_num = 22
정확도 :0.8090614886731392, seed_num = 23
정확도 :0.8317152103559871, seed_num = 24
정확도 :0.8466019417475729, seed_num = 

In [16]:
# seed = 42의 정확도 df 만들고 평균 확인 => 82.6
df_42 = pd.DataFrame.from_dict(dic_42, orient='index')
df_42.mean()

0    0.826731
dtype: float64

# (60.8) CuDNNLSTM / all fit / earlystop

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2

import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

# ---------------------
seed_num = 42
# ---------------------
tf.random.set_seed(seed_num)


def cudnn_lstm_model():
    model = Sequential()
    model.add(CuDNNLSTM(128, input_shape = (x.shape[1],x.shape[2]), return_sequences = True))
    model.add(Activation('hard_sigmoid'))
    model.add(CuDNNLSTM(units=64, return_sequences=True))
    model.add(Activation('hard_sigmoid'))
    model.add(Dropout(0.2))
    model.add(CuDNNLSTM(units=64, return_sequences=True))
    model.add(Activation('hard_sigmoid'))
    model.add(CuDNNLSTM(units=32, return_sequences=False))
    model.add(Activation('hard_sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])    
    
    return model

with tf.device('/device:GPU:1'):
    model = cudnn_lstm_model()

    from tensorflow.keras.callbacks import ModelCheckpoint
    import os

    MODEL_SAVE_FOLDER_PATH = './model/CuDNNLSTM/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    model_path = MODEL_SAVE_FOLDER_PATH + '4068_allfit-{epoch:02d}-{val_loss:.4f}.hdf5'

    cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                    verbose=1, save_best_only=True)

    early_stop = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, restore_best_weights=False)
    model.fit(x, y, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
46/46 [==============================] - 2s 24ms/step - loss: 0.6569 - accuracy: 0.6459 - val_loss: 0.7095 - val_accuracy: 0.4865

Epoch 00001: val_loss improved from inf to 0.70946, saving model to ./model/CuDNNLSTM/4068_allfit_p80-01-0.7095.hdf5
Epoch 2/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6757 - accuracy: 0.6088 - val_loss: 0.7536 - val_accuracy: 0.4865

Epoch 00002: val_loss did not improve from 0.70946
Epoch 3/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6652 - accuracy: 0.6385 - val_loss: 0.7482 - val_accuracy: 0.4865

Epoch 00003: val_loss did not improve from 0.70946
Epoch 4/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6636 - accuracy: 0.6433 - val_loss: 0.7505 - val_accuracy: 0.4865

Epoch 00004: val_loss did not improve from 0.70946
Epoch 5/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6631 - accuracy: 0.6406 - val_loss: 0.7496 - val_accuracy: 0.4865

Epoch 00005: va


Epoch 00042: val_loss did not improve from 0.70946
Epoch 43/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6545 - accuracy: 0.6513 - val_loss: 0.7476 - val_accuracy: 0.4865

Epoch 00043: val_loss did not improve from 0.70946
Epoch 44/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6533 - accuracy: 0.6513 - val_loss: 0.7472 - val_accuracy: 0.4865

Epoch 00044: val_loss did not improve from 0.70946
Epoch 45/500
46/46 [==============================] - 1s 11ms/step - loss: 0.6577 - accuracy: 0.6519 - val_loss: 0.7469 - val_accuracy: 0.4865

Epoch 00045: val_loss did not improve from 0.70946
Epoch 46/500
46/46 [==============================] - 1s 14ms/step - loss: 0.6528 - accuracy: 0.6504 - val_loss: 0.7479 - val_accuracy: 0.4865

Epoch 00046: val_loss did not improve from 0.70946
Epoch 47/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6533 - accuracy: 0.6509 - val_loss: 0.7479 - val_accuracy: 0.4865

Epoch 00047: val_loss di

In [15]:
# ----------------------------
seed_num = 42
# ----------------------------

from keras.models import load_model
best_model = load_model('./model/CuDNNLSTM/4068_allfit-01-0.7095.hdf5') # 수동으로 확인

dic={}
for seed in range(0, 50):
    random.seed(seed)

    x = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')
    
df = pd.DataFrame.from_dict(dic, orient='index')
print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.6045307443365696, seed_num = 0
정확도 :0.6051779935275081, seed_num = 1
정확도 :0.6110032362459547, seed_num = 2
정확도 :0.602588996763754, seed_num = 3
정확도 :0.6097087378640776, seed_num = 4
정확도 :0.6110032362459547, seed_num = 5
정확도 :0.6110032362459547, seed_num = 6
정확도 :0.5928802588996763, seed_num = 7
정확도 :0.6226537216828479, seed_num = 8
정확도 :0.6129449838187703, seed_num = 9
정확도 :0.5909385113268608, seed_num = 10
정확도 :0.6297734627831715, seed_num = 11
정확도 :0.601294498381877, seed_num = 12
정확도 :0.598705501618123, seed_num = 13
정확도 :0.6071197411003236, seed_num = 14
정확도 :0.6135922330097088, seed_num = 15
정확도 :0.6330097087378641, seed_num = 16
정확도 :0.598705501618123, seed_num = 17
정확도 :0.6045307443365696, seed_num = 18
정확도 :0.596116504854369, seed_num = 19
정확도 :0.6187702265372168, seed_num = 20
정확도 :0.6135922330097088, seed_num = 21
정확도 :0.6, seed_num = 22
정확도 :0.6032362459546926, seed_num = 23
정확도 :0.6032362459546926, seed_num = 24
정확도 :0.6071197411003236, seed_num = 25
정확도 :0.595469255

# (83.0) CuDNNLSTM / all fit / no earlystop

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2

import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

# ---------------------
seed_num = 42
# ---------------------
tf.random.set_seed(seed_num)


def cudnn_lstm_model():
    model = Sequential()
    model.add(CuDNNLSTM(128, input_shape = (x.shape[1],x.shape[2]), return_sequences = True))
    model.add(Activation('hard_sigmoid'))
    model.add(CuDNNLSTM(units=64, return_sequences=True))
    model.add(Activation('hard_sigmoid'))
    model.add(Dropout(0.2))
    model.add(CuDNNLSTM(units=64, return_sequences=True))
    model.add(Activation('hard_sigmoid'))
    model.add(CuDNNLSTM(units=32, return_sequences=False))
    model.add(Activation('hard_sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])    
    
    return model

with tf.device('/device:GPU:1'):
    model = cudnn_lstm_model()

    from tensorflow.keras.callbacks import ModelCheckpoint
    import os

    MODEL_SAVE_FOLDER_PATH = './model/CuDNNLSTM/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    model_path = MODEL_SAVE_FOLDER_PATH + '4068_allfit_noearly-{epoch:02d}-{val_loss:.4f}.hdf5'

    cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                    verbose=1, save_best_only=True)

#     early_stop = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, restore_best_weights=False)
    model.fit(x, y, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[cb_checkpoint], shuffle=False)

Epoch 1/500
46/46 [==============================] - 3s 22ms/step - loss: 0.6569 - accuracy: 0.6459 - val_loss: 0.7095 - val_accuracy: 0.4865

Epoch 00001: val_loss improved from inf to 0.70946, saving model to ./model/CuDNNLSTM/4068_allfit_noearly-01-0.7095.hdf5
Epoch 2/500
46/46 [==============================] - 1s 11ms/step - loss: 0.6757 - accuracy: 0.6088 - val_loss: 0.7536 - val_accuracy: 0.4865

Epoch 00002: val_loss did not improve from 0.70946
Epoch 3/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6652 - accuracy: 0.6385 - val_loss: 0.7482 - val_accuracy: 0.4865

Epoch 00003: val_loss did not improve from 0.70946
Epoch 4/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6636 - accuracy: 0.6433 - val_loss: 0.7505 - val_accuracy: 0.4865

Epoch 00004: val_loss did not improve from 0.70946
Epoch 5/500
46/46 [==============================] - 1s 11ms/step - loss: 0.6631 - accuracy: 0.6406 - val_loss: 0.7496 - val_accuracy: 0.4865

Epoch 00005


Epoch 00042: val_loss did not improve from 0.70946
Epoch 43/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6545 - accuracy: 0.6513 - val_loss: 0.7476 - val_accuracy: 0.4865

Epoch 00043: val_loss did not improve from 0.70946
Epoch 44/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6533 - accuracy: 0.6513 - val_loss: 0.7472 - val_accuracy: 0.4865

Epoch 00044: val_loss did not improve from 0.70946
Epoch 45/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6577 - accuracy: 0.6519 - val_loss: 0.7469 - val_accuracy: 0.4865

Epoch 00045: val_loss did not improve from 0.70946
Epoch 46/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6528 - accuracy: 0.6504 - val_loss: 0.7479 - val_accuracy: 0.4865

Epoch 00046: val_loss did not improve from 0.70946
Epoch 47/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6533 - accuracy: 0.6509 - val_loss: 0.7479 - val_accuracy: 0.4865

Epoch 00047: val_loss di

46/46 [==============================] - 1s 11ms/step - loss: 0.6511 - accuracy: 0.6519 - val_loss: 0.7470 - val_accuracy: 0.4865

Epoch 00085: val_loss did not improve from 0.70946
Epoch 86/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6529 - accuracy: 0.6518 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00086: val_loss did not improve from 0.70946
Epoch 87/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6511 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00087: val_loss did not improve from 0.70946
Epoch 88/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6515 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00088: val_loss did not improve from 0.70946
Epoch 89/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6508 - accuracy: 0.6519 - val_loss: 0.7470 - val_accuracy: 0.4865

Epoch 00089: val_loss did not improve from 0.70946
Epoch 90/500
46/46 [==================

46/46 [==============================] - 1s 12ms/step - loss: 0.6510 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00127: val_loss did not improve from 0.70946
Epoch 128/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6493 - accuracy: 0.6519 - val_loss: 0.7477 - val_accuracy: 0.4865

Epoch 00128: val_loss did not improve from 0.70946
Epoch 129/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6504 - accuracy: 0.6519 - val_loss: 0.7473 - val_accuracy: 0.4865

Epoch 00129: val_loss did not improve from 0.70946
Epoch 130/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6494 - accuracy: 0.6519 - val_loss: 0.7476 - val_accuracy: 0.4865

Epoch 00130: val_loss did not improve from 0.70946
Epoch 131/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6502 - accuracy: 0.6519 - val_loss: 0.7474 - val_accuracy: 0.4865

Epoch 00131: val_loss did not improve from 0.70946
Epoch 132/500
46/46 [=============

46/46 [==============================] - 1s 11ms/step - loss: 0.6490 - accuracy: 0.6519 - val_loss: 0.7473 - val_accuracy: 0.4865

Epoch 00169: val_loss did not improve from 0.70946
Epoch 170/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6495 - accuracy: 0.6519 - val_loss: 0.7467 - val_accuracy: 0.4865

Epoch 00170: val_loss did not improve from 0.70946
Epoch 171/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6493 - accuracy: 0.6519 - val_loss: 0.7466 - val_accuracy: 0.4865

Epoch 00171: val_loss did not improve from 0.70946
Epoch 172/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6501 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00172: val_loss did not improve from 0.70946
Epoch 173/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6494 - accuracy: 0.6519 - val_loss: 0.7472 - val_accuracy: 0.4865

Epoch 00173: val_loss did not improve from 0.70946
Epoch 174/500
46/46 [=============

...

46/46 [==============================] - 1s 13ms/step - loss: 0.6489 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00295: val_loss did not improve from 0.70946
Epoch 296/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6495 - accuracy: 0.6519 - val_loss: 0.7464 - val_accuracy: 0.4865

Epoch 00296: val_loss did not improve from 0.70946
Epoch 297/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6489 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00297: val_loss did not improve from 0.70946
Epoch 298/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6490 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00298: val_loss did not improve from 0.70946
Epoch 299/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6486 - accuracy: 0.6519 - val_loss: 0.7471 - val_accuracy: 0.4865

Epoch 00299: val_loss did not improve from 0.70946
Epoch 300/500
46/46 [=============

46/46 [==============================] - 1s 12ms/step - loss: 0.6493 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00337: val_loss did not improve from 0.70946
Epoch 338/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6481 - accuracy: 0.6519 - val_loss: 0.7441 - val_accuracy: 0.4865

Epoch 00338: val_loss did not improve from 0.70946
Epoch 339/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6451 - accuracy: 0.6519 - val_loss: 0.7390 - val_accuracy: 0.4865

Epoch 00339: val_loss did not improve from 0.70946
Epoch 340/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6382 - accuracy: 0.6519 - val_loss: 0.7413 - val_accuracy: 0.4865

Epoch 00340: val_loss did not improve from 0.70946
Epoch 341/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6496 - accuracy: 0.6519 - val_loss: 0.7490 - val_accuracy: 0.4865

Epoch 00341: val_loss did not improve from 0.70946
Epoch 342/500
46/46 [=============

46/46 [==============================] - 1s 13ms/step - loss: 0.2788 - accuracy: 0.9217 - val_loss: 0.7020 - val_accuracy: 0.7143

Epoch 00416: val_loss did not improve from 0.65264
Epoch 417/500
46/46 [==============================] - 1s 12ms/step - loss: 0.2674 - accuracy: 0.9253 - val_loss: 0.7079 - val_accuracy: 0.7117

Epoch 00417: val_loss did not improve from 0.65264
Epoch 418/500
46/46 [==============================] - 1s 13ms/step - loss: 0.2734 - accuracy: 0.9258 - val_loss: 0.7094 - val_accuracy: 0.6988

Epoch 00418: val_loss did not improve from 0.65264
Epoch 419/500
46/46 [==============================] - 1s 12ms/step - loss: 0.2757 - accuracy: 0.9194 - val_loss: 0.7374 - val_accuracy: 0.6988

Epoch 00419: val_loss did not improve from 0.65264
Epoch 420/500
46/46 [==============================] - 1s 13ms/step - loss: 0.2833 - accuracy: 0.9180 - val_loss: 0.7111 - val_accuracy: 0.7076

Epoch 00420: val_loss did not improve from 0.65264
Epoch 421/500
46/46 [=============

46/46 [==============================] - 1s 13ms/step - loss: 0.2070 - accuracy: 0.9531 - val_loss: 0.8393 - val_accuracy: 0.7096

Epoch 00458: val_loss did not improve from 0.65264
Epoch 459/500
46/46 [==============================] - 1s 12ms/step - loss: 0.2045 - accuracy: 0.9501 - val_loss: 0.8363 - val_accuracy: 0.7138

Epoch 00459: val_loss did not improve from 0.65264
Epoch 460/500
46/46 [==============================] - 1s 13ms/step - loss: 0.2018 - accuracy: 0.9519 - val_loss: 0.8892 - val_accuracy: 0.6900

Epoch 00460: val_loss did not improve from 0.65264
Epoch 461/500
46/46 [==============================] - 1s 12ms/step - loss: 0.2014 - accuracy: 0.9546 - val_loss: 0.8708 - val_accuracy: 0.7055

Epoch 00461: val_loss did not improve from 0.65264
Epoch 462/500
46/46 [==============================] - 1s 13ms/step - loss: 0.1996 - accuracy: 0.9538 - val_loss: 0.8744 - val_accuracy: 0.7008

Epoch 00462: val_loss did not improve from 0.65264
Epoch 463/500
46/46 [=============

46/46 [==============================] - 0s 11ms/step - loss: 0.1690 - accuracy: 0.9620 - val_loss: 1.0511 - val_accuracy: 0.6827

Epoch 00500: val_loss did not improve from 0.65264


In [5]:
from sklearn import metrics 

import random
# ----------------------------
seed_num = 42
# ----------------------------

with tf.device('/device:GPU:1'):

    from keras.models import load_model
    best_model = load_model('./model/CuDNNLSTM/4068_allfit_noearly-386-0.6526.hdf5') # 수동으로 확인

    dic={}
    for seed in range(0, 50):
        random.seed(seed)

        x = np.load('/project/LSH/x_(7727,10,4068).npy')
        y = np.load('/project/LSH/y_(7727,1).npy')

        idx = list(range(len(x)))
        random.shuffle(idx)

        i = round(x.shape[0]*0.8)
        X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
        X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

        pred = best_model.predict(X_test)
        pred[pred>0.5]=1
        pred[pred<=0.5]=0
        acc = metrics.accuracy_score(y_test, pred)
        dic[seed]=acc
        print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

    df = pd.DataFrame.from_dict(dic, orient='index')
    print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.8271844660194175, seed_num = 0
정확도 :0.83042071197411, seed_num = 1
정확도 :0.8355987055016181, seed_num = 2
정확도 :0.8330097087378641, seed_num = 3
정확도 :0.829126213592233, seed_num = 4
정확도 :0.8394822006472492, seed_num = 5
정확도 :0.8343042071197411, seed_num = 6
정확도 :0.8414239482200647, seed_num = 7
정확도 :0.8284789644012945, seed_num = 8
정확도 :0.8194174757281554, seed_num = 9
정확도 :0.8362459546925566, seed_num = 10
정확도 :0.827831715210356, seed_num = 11
정확도 :0.8375404530744337, seed_num = 12
정확도 :0.8440129449838187, seed_num = 13
정확도 :0.8271844660194175, seed_num = 14
정확도 :0.83042071197411, seed_num = 15
정확도 :0.8258899676375404, seed_num = 16
정확도 :0.8135922330097087, seed_num = 17
정확도 :0.8265372168284789, seed_num = 18
정확도 :0.8517799352750809, seed_num = 19
정확도 :0.8349514563106796, seed_num = 20
정확도 :0.8097087378640777, seed_num = 21
정확도 :0.8252427184466019, seed_num = 22
정확도 :0.8297734627831715, seed_num = 23
정확도 :0.8245954692556634, seed_num = 24
정확도 :0.8368932038834952, seed_num = 25
정확

# (76.7) CuDNNLSTM / not all fit

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2

import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# ---------------------
seed_num = 42
# ---------------------
tf.random.set_seed(seed_num)


def cudnn_lstm_model():
    model = Sequential()
    model.add(CuDNNLSTM(128, input_shape = (X_train.shape[1],X_train.shape[2]), return_sequences = True))
    model.add(Activation('hard_sigmoid'))
    model.add(CuDNNLSTM(units=64, return_sequences=True))
    model.add(Activation('hard_sigmoid'))
    model.add(Dropout(0.2))
    model.add(CuDNNLSTM(units=64, return_sequences=True))
    model.add(Activation('hard_sigmoid'))
    model.add(CuDNNLSTM(units=32, return_sequences=False))
    model.add(Activation('hard_sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])    
    
    return model

model = cudnn_lstm_model()


from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/CuDNNLSTM/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + '4068_notallfit-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, restore_best_weights=False)
model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 2s 23ms/step - loss: 0.6849 - accuracy: 0.5887 - val_loss: 0.6655 - val_accuracy: 0.6177

Epoch 00001: val_loss improved from inf to 0.66551, saving model to ./model/CuDNNLSTM/4068_notallfit-01-0.6655.hdf5
Epoch 2/500
37/37 [==============================] - 0s 11ms/step - loss: 0.6810 - accuracy: 0.5988 - val_loss: 0.6652 - val_accuracy: 0.6177

Epoch 00002: val_loss improved from 0.66551 to 0.66520, saving model to ./model/CuDNNLSTM/4068_notallfit-02-0.6652.hdf5
Epoch 3/500
37/37 [==============================] - 0s 13ms/step - loss: 0.6814 - accuracy: 0.5951 - val_loss: 0.6652 - val_accuracy: 0.6177

Epoch 00003: val_loss did not improve from 0.66520
Epoch 4/500
37/37 [==============================] - 0s 11ms/step - loss: 0.6772 - accuracy: 0.5988 - val_loss: 0.6652 - val_accuracy: 0.6177

Epoch 00004: val_loss improved from 0.66520 to 0.66517, saving model to ./model/CuDNNLSTM/4068_notallfit-04-0.6652.hdf5
Epoch 5/500
37/37 [==

37/37 [==============================] - 0s 13ms/step - loss: 0.3132 - accuracy: 0.9085 - val_loss: 0.6783 - val_accuracy: 0.7277

Epoch 00039: val_loss did not improve from 0.58819
Epoch 40/500
37/37 [==============================] - 0s 12ms/step - loss: 0.2987 - accuracy: 0.9146 - val_loss: 0.6857 - val_accuracy: 0.7219

Epoch 00040: val_loss did not improve from 0.58819
Epoch 41/500
37/37 [==============================] - 0s 13ms/step - loss: 0.3005 - accuracy: 0.9161 - val_loss: 0.6966 - val_accuracy: 0.7225

Epoch 00041: val_loss did not improve from 0.58819
Epoch 42/500
37/37 [==============================] - 0s 11ms/step - loss: 0.2960 - accuracy: 0.9165 - val_loss: 0.6935 - val_accuracy: 0.7309

Epoch 00042: val_loss did not improve from 0.58819
Epoch 43/500
37/37 [==============================] - 0s 11ms/step - loss: 0.2884 - accuracy: 0.9195 - val_loss: 0.7048 - val_accuracy: 0.7225

Epoch 00043: val_loss did not improve from 0.58819
Epoch 44/500
37/37 [==================

In [12]:
# ----------------------------
seed_num = 42
# ----------------------------

from keras.models import load_model
best_model = load_model('./model/CuDNNLSTM/4068_notallfit-15-0.5882.hdf5') # 수동으로 확인

dic={}
for seed in range(0, 50):
    random.seed(seed)

    x = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')
    
df = pd.DataFrame.from_dict(dic, orient='index')
print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.7508090614886731, seed_num = 0
정확도 :0.772168284789644, seed_num = 1
정확도 :0.7708737864077669, seed_num = 2
정확도 :0.7844660194174757, seed_num = 3
정확도 :0.7579288025889968, seed_num = 4
정확도 :0.7792880258899676, seed_num = 5
정확도 :0.7656957928802589, seed_num = 6
정확도 :0.7695792880258899, seed_num = 7
정확도 :0.7728155339805826, seed_num = 8
정확도 :0.7611650485436893, seed_num = 9
정확도 :0.7702265372168284, seed_num = 10
정확도 :0.7773462783171521, seed_num = 11
정확도 :0.7637540453074434, seed_num = 12
정확도 :0.7624595469255664, seed_num = 13
정확도 :0.7669902912621359, seed_num = 14
정확도 :0.7792880258899676, seed_num = 15
정확도 :0.7708737864077669, seed_num = 16
정확도 :0.7702265372168284, seed_num = 17
정확도 :0.7682847896440129, seed_num = 18
정확도 :0.7747572815533981, seed_num = 19
정확도 :0.7669902912621359, seed_num = 20
정확도 :0.7611650485436893, seed_num = 21
정확도 :0.7877022653721683, seed_num = 22
정확도 :0.772168284789644, seed_num = 23
정확도 :0.7501618122977346, seed_num = 24
정확도 :0.7682847896440129, seed_num = 2